## Библиотеки 

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import random
from random import randint

import warnings # ?
warnings.filterwarnings("ignore") # убираем оповещения о предупреждениях. Не засоряем вывод
print('Let me Die1')

Let me Die1


## Получение данных

In [2]:
df_train = pd.read_csv('F:\Anac_python\Examples_Python\Jupyter_Notebook_stuff\Alpha_test\\alfabank-campus\df_train.csv', sep=';') # грузим скаченный файл csv с данными для тренинга в переменную
df_test = pd.read_csv('F:\Anac_python\Examples_Python\Jupyter_Notebook_stuff\Alpha_test\\alfabank-campus\df_test.csv', sep=';')   # грузим скаченный файл csv с даннымми для проверки тренировки в переменную

In [3]:
df_train['Data'] = df_train.Data.apply(lambda s: list(map(int, s.split(',')))) # Изначально в файле только 3 столбца "id", "Data" и "Target". Тянем в Data данные для обучения
df_train['Target'] = df_train.Target.apply(lambda s: list(map(int, s.split(',')))) # Изначально в файле только 3 столбца "id", "Data" и "Target". Тянем в Target ответы для тренировки
df_test['Data'] = df_test.Data.apply(lambda s: list(map(int, s.split(',')))) # Изначально в файле только 2 "id" и "Data" . Тянем в Data данные для отработки модели на точность

In [4]:
df_train.head()

,Id,Data,Target
0,0,"[4814, 4814, 6010, 6011, 4814, 6011, 6011, 481...","[4814, 4814, 4814, 4814, 5411, 4814, 4814, 481..."
1,1,"[6011, 6011, 6011, 6011, 6011, 6011, 6011, 481...","[4814, 6011, 4814, 6011, 4814, 4814, 6011, 481..."
2,2,"[8021, 6011, 6011, 6010, 4829, 4814, 6011, 601...","[6011, 6011, 6010, 4829, 4829, 6010, 6011, 601..."
3,3,"[4814, 6011, 4814, 4814, 4814, 6011, 6011, 569...","[6011, 6011, 6010, 6011, 6011, 4814, 4814, 601..."
4,4,"[4814, 4814, 4814, 4814, 4814, 4814, 5946, 481...","[5499, 6011, 4814, 4829, 5200, 5411, 5499, 591..."


In [5]:
df_train.shape

(7033, 3)

In [6]:
df_train.dtypes

Id         int64
Data      object
Target    object
dtype: object

In [7]:
df_test.head()

,Id,Data
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482..."
3,3,"[6011, 4722, 4722, 4722, 4814, 6011, 6011, 482..."
4,4,"[4814, 4814, 4814, 6011, 4814, 4814, 4814, 481..."


In [8]:
df_test.shape

(7033, 2)

In [9]:
df_test.dtypes

Id       int64
Data    object
dtype: object

## Функции (чё?)

In [10]:
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted) > k: # тянем первые k значений
        predicted = predicted[:k]

    score = 0.0    # общая сумма точности
    num_hits = 0.0 # для подсчета числа верно предсказанных элементов. 

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

## Baseline 1: топ10 MCC-кодов из train-части

MCC-код и соответствующее количество вхождений в train-часть

In [11]:
# Разбиваем explode'ом список на несколько строк с одним значением.
# Считаем уникальные значения списке
# Берем первые 10 уникальных значений по убыванию. От самых частых и популярных в сторону уменьшения популярности
top10_codes = df_train['Data'].explode().value_counts().head(10) 
# Выводи наш "топ10 аниме предательств" для наглядности
top10_codes

6011    700677
6010    490602
4814    473396
5411    472408
4829    307388
5499    164719
5541     68224
5912     65071
5331     61833
5812     52029
Name: Data, dtype: int64

In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7033 entries, 0 to 7032
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      7033 non-null   int64 
 1   Data    7033 non-null   object
 2   Target  7033 non-null   object
dtypes: int64(1), object(2)
memory usage: 165.0+ KB


In [20]:
mapk(df_train['Target'], [top10_codes.index]*len(df_train)) # применяем магию математики. Тут надо подкуриться/поспать
# df_train['Target']
# [top10_codes.index]*len(df_train) делает 7033 списка с длиной 10
# и они содержат в себе топ 10 самых популярных MCC кодов



0.2742435829727881

In [24]:
df_train["Data"].map(len).describe([0.5,0.75,0.9,0.95,0.99])

count     7033.000000
mean       473.322906
std        811.665063
min         40.000000
50%        336.000000
75%        570.000000
90%        872.000000
95%       1113.000000
99%       2156.040000
max      21101.000000
Name: Data, dtype: float64

In [23]:
df_test["Data"].map(len).describe([0.5,0.75,0.9,0.95,0.99]) # для KAGGLE

count     7033.000000
mean       476.756150
std       1341.916565
min         40.000000
50%        344.000000
75%        582.000000
90%        867.000000
95%       1108.000000
99%       1916.680000
max      88771.000000
Name: Data, dtype: float64

## Baseline 2: cамые популярные транзакции пользователя.

**   
1. Из массива датасета сделать дата фрейм: такой что
 - 2000 стоблцов по типу MCC_1 MCC_2 и по MCC_2000
 - если у чела меньше 2000 транзакций и всего типа 30-40-1999 то пустые дырки забиваем значением "копия_значения_1, копия_значения_2, копия_значения_3, копия_значения_4, 1,2,3,4" 
 -  если у чела больше 2000 транзакций то хапаем в таблицу ПОСЛЕДНИЕ 2000 транзакций так как это более релевантные, епта 
 - 10 столбцов предиктов. 1 2 3 4 5 6 и такое всё
--! ПЕРСУШНЫЕ АЙДИШНИКИ НЕ ПИХАЕМ В НЕГО, а машина начнет эту херь мешать с MCC кодами -_-

2. RandomForest траим
Катбуст научился чет небалансные деревья крутить мб он ок

Сначала выберем - catboost
Потом попробуем - LightGBM
может даже трайнуть такую - xgboost

И им надо метрику прикрутить (AP@K при K=10) для 10ти столбцов с 30% Target'ом что бы сравнивать выход и желаемые значения!

Это для тренинга
Х - че влетает на вход. MCC коды 
Y - Ответы для ИКСОВ. 
Таблички Y и X имеют одинаковое кол-во строк и они не подлежат всяким сортировкам-перетиасовкам, епт
Тут гайд подсказка:
( https://stackoverflow.com/questions/50564167/xgboost-split-data-in-train-and-test )

Надо выбрать агрегатор для 7000 человек-юзверей для 7000 AP@K'ов ВСТРОЕННОЙ ФУНКЦИЕЙ ПОЛУЧИМ MAP@K 
используем СРЕДНЕЕ ЗНАЧЕНИЕ, а не медианое (мб надо проверить этот момент)

Косяки: модель может допереть что чтьо-то ест ьиз паттернов повторюхи для коротких челиксов! Но это маловероятно вроде быыыы ы
Может быть косяк с тем, что модель полюбит рандомное значение (LICKeage). Но в жизни такая штука не будет МАССОВОЙ.  

Ликов надо чистить по сути, с опытом понимаешь че как с ней.
Читануть тут: https://en.wikipedia.org/wiki/Leakage_(machine_learning)

**

In [25]:
df_train['pred_baseline_2'] = df_train['Data'].apply(get_top_codes)

In [16]:
mapk(df_train['Target'], df_train['pred_baseline_2'])

0.3236094127683776

## Submission

предсказания из второго бейзлайна

In [17]:
df_test['Predicted'] = df_test['Data'].apply(get_top_codes)

In [18]:
df_test.head()

,Id,Data,Predicted
0,0,"[4814, 4814, 6011, 6011, 6010, 6011, 6011, 481...","[6011, 4814, 6010, 4829, 5499, 5541, 5411, 601..."
1,1,"[6010, 6011, 6010, 5411, 5411, 5977, 6011, 601...","[6011, 6010, 5411, 6011, 6010, 4814, 5411, 482..."
2,2,"[4814, 6011, 5251, 6011, 7832, 5641, 5814, 482...","[6011, 6010, 5499, 4814, 5411, 5814, 4829, 581..."
3,3,"[6011, 4722, 4722, 4722, 4814, 6011, 6011, 482...","[6011, 4814, 5964, 5411, 4829, 5912, 6010, 601..."
4,4,"[4814, 4814, 4814, 6011, 4814, 4814, 4814, 481...","[4814, 6011, 6011, 6010, 4814, 5411, 4829, 549..."


In [19]:
submission_baseline_2 = df_test[['Id', 'Predicted']]
submission_baseline_2['Predicted'] = submission_baseline_2.Predicted.astype(str).str.replace(',', '')
submission_baseline_2.to_csv('submission_baseline_2.csv', index=False)